In [ ]:
#Data with certain noise

#Finding the high density food environment

x_coordinates = [0]*len(coordinates)
y_coordinates = [0]*len(coordinates)
for i in range(len(coordinates)):
    x_coordinates[i] = coordinates[i][0]
    y_coordinates[i] = coordinates[i][1]
    

import itertools
x_limits = [(0,24),(25,49),(50,74), (75,99)]
y_limits = [(0,24),(25,49),(50,74), (75,99)]

combos = list(itertools.product(x_limits, y_limits))
density  = [] 
for i in range(len(x_limits)):
    
    xx_lim_do = x_limits[i][0]
    xx_lim_up = x_limits[i][1]

    xx =  np.where((np.array(x_coordinates) <= xx_lim_up) & (np.array(x_coordinates)  >= xx_lim_do))
    
    for j in range(len(y_limits)):
        
        yy_lim_do = y_limits[j][0]
        yy_lim_up = y_limits[j][1]
        
        yy =  np.where((np.array(y_coordinates) <= yy_lim_up) & (np.array(y_coordinates)  >= yy_lim_do))
        
        density.append(len(np.intersect1d(xx , yy)))
        
np.where(density == np.max(density))[0][0]  

HDsection = combos[np.where(density == np.max(density))[0][0] ]


xx_lim_do = HDsection[0][0]
xx_lim_up = HDsection[0][1]
yy_lim_do = HDsection[1][0]
yy_lim_up = HDsection[1][1]

xx = np.where((np.array(x_coordinates) < xx_lim_up) & (np.array(x_coordinates)  > xx_lim_do))
yy = np.where((np.array(y_coordinates) < yy_lim_up) & (np.array(y_coordinates)  > yy_lim_do))

sample_food = np.random.choice(np.intersect1d(xx , yy) ,1)[0]